In [1]:
from bs4 import BeautifulSoup
import urllib.request
import requests
#from urllib.request import Request

In [2]:
seed_url = "https://www.europarl.europa.eu/news/en/press-room"

urls = [seed_url]    #queue of urls to crawl
seen = [seed_url]    #stack of urls seen so far
opened = []          #we keep track of seen urls so that we don't revisit them
crisis_related = []


In [3]:
maxNumUrl = 500; #set the maximum number of urls to visit
print("Starting with url="+str(urls))
while len(urls) > 0 and len(opened) < maxNumUrl and len(crisis_related) < 10:
    try:
        curr_url = urls.pop(0)
        print("num. of URLs in stack: %d " % len(urls))
        print("Trying to access= "+curr_url)
        req = urllib.request.Request(curr_url,headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urllib.request.urlopen(req).read()
        opened.append(curr_url)
    except Exception as ex:
        print("Unable to access= "+curr_url)
        print(ex)
        continue
    
    #Part 1: check curr_url and see if it is a crisis_related press release article
    soup = BeautifulSoup(webpage)
    text = soup.get_text().lower().replace('-',' ').replace(',',' ').replace('.',' ').replace('\n',' ').split()
    find_p= False
    for tag in soup.find_all('span',class_="ep_name"): #check if curr_url is a Plenary session article
        ep_name= tag.text
        if ep_name=='Plenary session' and soup.find('article',id= 'website-body'):
            find_p= True
    if 'crisis' in text and find_p:
        crisis_related.append(curr_url)
    print("num of crisis urls: %d" % len(crisis_related))
    
    #Part2: keep crawling and saving all children href into url if they are new and consistent with seed
    for tag in soup.find_all('a', href=True): #find tags with links
        childUrl = tag['href'] #extract just the link
        childUrl = urllib.parse.urljoin(seed_url, childUrl)
        if seed_url in childUrl and childUrl not in seen:
            urls.append(childUrl)
            seen.append(childUrl)

print("num. of URLs seen = %d, and scanned = %d" % (len(seen), len(opened)))
#process time is around 4 min

Starting with url=['https://www.europarl.europa.eu/news/en/press-room']
num. of URLs in stack: 0 
Trying to access= https://www.europarl.europa.eu/news/en/press-room
num of crisis urls: 0
num. of URLs in stack: 28 
Trying to access= https://www.europarl.europa.eu/news/en/press-room#website-body
num of crisis urls: 0
num. of URLs in stack: 27 
Trying to access= https://www.europarl.europa.eu/news/en/press-room#language-select
num of crisis urls: 0
num. of URLs in stack: 26 
Trying to access= https://www.europarl.europa.eu/news/en/press-room#search-field
num of crisis urls: 0
num. of URLs in stack: 25 
Trying to access= https://www.europarl.europa.eu/news/en/press-room/accreditation
num of crisis urls: 0
num. of URLs in stack: 28 
Trying to access= https://www.europarl.europa.eu/news/en/press-room/press-tool-kit
num of crisis urls: 0
num. of URLs in stack: 39 
Trying to access= https://www.europarl.europa.eu/news/en/press-room/contacts
num of crisis urls: 0
num. of URLs in stack: 49 
Try

C:\Users\jimmy\anaconda3\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


num of crisis urls: 1
num. of URLs in stack: 109 
Trying to access= https://twitter.com/intent/tweet?text=Press%20room%20%7C%20News%20%7C%20European%20Parliament&url=https://www.europarl.europa.eu/news/en/press-room?xtor%3DAD-78-%5BSocial_share_buttons%5D-%5Btwitter%5D-%5Ben%5D-%5Bnews%5D-%5Bpressroom%5D-%5Bhomepage_pressroom%5D-&via=Europarl_EN
Unable to access= https://twitter.com/intent/tweet?text=Press%20room%20%7C%20News%20%7C%20European%20Parliament&url=https://www.europarl.europa.eu/news/en/press-room?xtor%3DAD-78-%5BSocial_share_buttons%5D-%5Btwitter%5D-%5Ben%5D-%5Bnews%5D-%5Bpressroom%5D-%5Bhomepage_pressroom%5D-&via=Europarl_EN
HTTP Error 302: The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Found
num. of URLs in stack: 108 
Trying to access= https://www.linkedin.com/shareArticle?mini=true&url=https://www.europarl.europa.eu/news/en/press-room?xtor%3DAD-78-%5BSocial_share_buttons%5D-%5Blinkedin%5D-%5Ben%5D-%5Bnews%5

In [4]:
print("List of URLs that contains crisis:")
for url in crisis_related:
    print(url)

List of URLs that contains crisis:
https://www.europarl.europa.eu/news/en/press-room/20230929IPR06132/nagorno-karabakh-meps-demand-review-of-eu-relations-with-azerbaijan
https://www.europarl.europa.eu/news/en/press-room/20221209IPR64426/eu-long-term-budget-needs-urgent-revision-to-cope-with-current-crises
https://www.europarl.europa.eu/news/en/press-room/20210304IPR99207/parliament-gives-green-light-for-new-eu4health-programme
https://www.europarl.europa.eu/news/en/press-room/20220909IPR40138/parliament-adopts-new-rules-on-adequate-minimum-wages-for-all-workers-in-the-eu
https://www.europarl.europa.eu/news/en/press-room/20230310IPR77232/minimum-income-schemes-increasing-support-accessibility-and-inclusion
https://www.europarl.europa.eu/news/en/press-room/20230210IPR74806/green-deal-industrial-plan-securing-the-eu-s-clean-tech-leadership
https://www.europarl.europa.eu/news/en/press-room/20230707IPR02421/parliament-adopts-new-rules-to-boost-energy-savings
https://www.europarl.europa.eu/n

In [5]:
part=2
num=1 #num of url
for url in crisis_related:
    res = requests.get(url)
    html_content = res.text
    f = open("{}_{}.txt".format(part,num), "w", encoding= 'utf-8')
    f.write(html_content)
    f.close()
    print("HTML content for Question {}, article {} is saved.".format(part,num))
    num+=1

HTML content for Question 2, article 1 is saved.
HTML content for Question 2, article 2 is saved.
HTML content for Question 2, article 3 is saved.
HTML content for Question 2, article 4 is saved.
HTML content for Question 2, article 5 is saved.
HTML content for Question 2, article 6 is saved.
HTML content for Question 2, article 7 is saved.
HTML content for Question 2, article 8 is saved.
HTML content for Question 2, article 9 is saved.
HTML content for Question 2, article 10 is saved.
